In [1]:
import os 
from dotenv import load_dotenv 
load_dotenv()
groq_api_key=os.getenv('GROQ_API_KEY')

In [4]:
from langchain_groq import ChatGroq 
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)

In [5]:
from langchain_core.messages import HumanMessage 
model.invoke([HumanMessage(content="Hi, My name is sudeep and I am a AI engineer")])

AIMessage(content="Hello Sudeep, it's nice to meet you!\n\nAs an AI, I don't have personal experiences or a job title like you, but I'm always eager to learn more about what AI engineers do. \n\nWhat kind of AI projects are you working on?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 22, 'total_tokens': 83, 'completion_time': 0.110909091, 'prompt_time': 0.002133985, 'queue_time': 0.240833615, 'total_time': 0.113043076}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--020e8d93-6a5e-46c0-9e40-43f7b85d2163-0', usage_metadata={'input_tokens': 22, 'output_tokens': 61, 'total_tokens': 83})

In [6]:
from langchain_core.messages import AIMessage 
model.invoke(
    [
        HumanMessage(content="Hi, My name is sudeep and I am a AI engineer"),
        AIMessage(content="Hello Sudeep, it's nice to meet you!\n\nAs an AI, I don't have personal experiences or a job title like you, but I'm always eager to learn more about what AI engineers do. \n\nWhat kind of AI projects are you working on?\n"),
        HumanMessage(content="Hey What's my name and what do i do?")
    ]
)

AIMessage(content="You are Sudeep, and you are an AI engineer!  \n\nRemember, I don't have access to real-time information or personal details unless you choose to share them with me.  😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 103, 'total_tokens': 149, 'completion_time': 0.083636364, 'prompt_time': 0.005170678, 'queue_time': 0.24700893100000001, 'total_time': 0.088807042}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--1f6f69d7-8bef-4b3c-9121-39f6275edc23-0', usage_metadata={'input_tokens': 103, 'output_tokens': 46, 'total_tokens': 149})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory 
from langchain_core.runnables.history import RunnableWithMessageHistory
store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history=get_session_history)

In [8]:
config={"configurable":{"session_id":"chat1"}}

In [14]:
response=with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)

In [15]:
response.content

'Your name is Sudeep.  I remember from our earlier conversation!  😊  How can I help you today? \n\n\n\n'

In [16]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config1
)

In [17]:
response.content

'As an AI, I have no memory of past conversations and do not know your name. Please tell me your name so I can address you properly.\n'

### Prompt templates 
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input basides just the messages.

In [19]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain=prompt|model

In [20]:
chain.invoke({"messages":[HumanMessage(content='Hi My name is sudeep')]})

AIMessage(content="Hello Sudeep!\n\nIt's nice to meet you. How can I help you today?  \n\nI'll do my best to answer all your questions. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 31, 'total_tokens': 71, 'completion_time': 0.072727273, 'prompt_time': 0.002343227, 'queue_time': 0.246351192, 'total_time': 0.0750705}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--9bb0408a-e008-4bd4-9bee-69821199c7d8-0', usage_metadata={'input_tokens': 31, 'output_tokens': 40, 'total_tokens': 71})

In [21]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history=get_session_history)

In [22]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="My name is sudeep")],
    config=config
)

In [23]:
response.content

"Hello Sudeep! It's nice to meet you.\n\nI'm ready to answer your questions to the best of my ability. \n\nWhat can I help you with today?  😊  \n\n"

In [24]:
response=with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config=config
)

In [25]:
response.content

'Your name is Sudeep.  😊 \n\nI remember! How can I help you further?\n'

In [27]:
## Add more complexity 
prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {languages}"
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain=prompt|model

In [34]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is sudeep")],"languages":"Hindi"})

In [35]:
response.content

'नमस्ते सुदीप! 👋  \n\nआपका स्वागत है!  😊  मुझे आपकी मदद करने में खुशी होगी। आप मुझसे कोई भी सवाल पूछ सकते हैं, मैं अपनी पूरी कोशिश करूँगा कि आपको सही उत्तर दूँ।  \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [36]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key='messages'
)

In [37]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="My name is sudeep")],"languages":"Hindi"},
    config=config
)
response.content

'नमस्ते सुदीप! \n\nमैं आपकी मदद करने के लिए तैयार हूँ। कोई भी सवाल पूछने में संकोच न करें, मैं अपनी पूरी कोशिश करूँगा! 😊\n\n'

### Managing the Conversaion History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system messages and whether to allow partial messages

In [42]:
from langchain_core.messages import SystemMessage,trim_messages 
trimmer=trim_messages(
    max_tokens=45,
    strategy='last',
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on='human'
)

In [43]:
messages=[
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="what 2+2"),
    AIMessage(content="4"),
    HumanMessage(content='thanks'),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={})]

In [45]:
from operator import itemgetter 
from langchain_core.runnables import RunnablePassthrough
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model
)
response=chain.invoke({"messages":messages+[HumanMessage(content="What ice cream do i like")],
              "languages":"English"})
response.content

"As a large language model, I don't have access to your personal information, including your taste preferences like your favorite ice cream flavor. \n\nWhat's your favorite ice cream flavor? 😊  Maybe I can tell you some fun facts about it!\n"

In [46]:
response=chain.invoke({"messages":messages+[HumanMessage(content="What math problem did i ask")],
              "languages":"English"})
response.content

'You asked "what is 2+2". 🧮  \n\n\n\nLet me know if you\'d like to try another one! 😊\n'

In [47]:
## Lets wrap this is the message history 
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config={"configurable":{"session_id":"chat5"}}

In [50]:
response=with_message_history.invoke(
    {
        "messages":messages+[HumanMessage(content="Whats was my last question?")],
        "languages":"English"
    },
    config=config
)
response.content

'Your last question was "having fun?". 😊  \n\nIs everything okay?\n'